***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [1]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-0ef9  GCE       4                                       RUNNING  us-central1-a


# Imports & Setup

In [2]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [3]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Jan  9 07:21 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [5]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [6]:
spark

In [7]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'ass-3-bucket-tamar' 
client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    print(b.name)

dl_dict.pkl
dl_dict_STTEMING.pkl
graphframes.sh
page_view_dict.pkl
postings_gcp/0_000.bin
postings_gcp/0_001.bin
postings_gcp/0_002.bin
postings_gcp/0_003.bin
postings_gcp/0_004.bin
postings_gcp/0_005.bin
postings_gcp/0_006.bin
postings_gcp/0_007.bin
postings_gcp/0_008.bin
postings_gcp/0_009.bin
postings_gcp/0_010.bin
postings_gcp/0_011.bin
postings_gcp/0_012.bin
postings_gcp/0_013.bin
postings_gcp/0_014.bin
postings_gcp/0_015.bin
postings_gcp/0_016.bin
postings_gcp/0_017.bin
postings_gcp/0_018.bin
postings_gcp/0_019.bin
postings_gcp/0_020.bin
postings_gcp/0_021.bin
postings_gcp/0_022.bin
postings_gcp/0_023.bin
postings_gcp/0_024.bin
postings_gcp/0_025.bin
postings_gcp/0_026.bin
postings_gcp/0_027.bin
postings_gcp/0_028.bin
postings_gcp/0_posting_locs.pickle
postings_gcp/100_000.bin
postings_gcp/100_001.bin
postings_gcp/100_002.bin
postings_gcp/100_003.bin
postings_gcp/100_004.bin
postings_gcp/100_005.bin
postings_gcp/100_006.bin
postings_gcp/100_007.bin
postings_gcp/100_008.bin
postin

***GCP setup is complete!*** If you got here without any errors you've earned 10 out of the 35 points of this part.

# Building an inverted index

Here, we read the entire corpus to an rdd, directly from Google Storage Bucket and use your code from Colab to construct an inverted index.

In [ ]:
full_path = "gs://wikidata_preprocessed/*"
parquetFile = spark.read.parquet(full_path)
doc_text_pairs = parquetFile.select("text", "id").rdd

We will count the number of pages to make sure we are looking at the entire corpus. The number of pages should be more than 6M

In [ ]:
# Count number of wiki pages
parquetFile.count()

Let's import the inverted index module. Note that you need to use the staff-provided version called `inverted_index_gcp.py`, which contains helper functions to writing and reading the posting files similar to the Colab version, but with writing done to a Google Cloud Storage bucket.

In [ ]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_stemming_gcp.py

In [ ]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_stemming_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [ ]:
from inverted_index_stemming_gcp import *

**YOUR TASK (10 POINTS)**: Use your implementation of `word_count`, `reduce_word_counts`, `calculate_df`, and `partition_postings_and_write` functions from Colab to build an inverted index for all of English Wikipedia in under 2 hours.

A few notes: 
1. The number of corpus stopwords below is a bit bigger than the colab version since we are working on the whole corpus and not just on one file.
2. You need to slightly modify your implementation of  `partition_postings_and_write` because the signature of `InvertedIndex.write_a_posting_list` has changed and now includes an additional argument called `bucket_name` for the target bucket. See the module for more details.
3. You are not allowed to change any of the code not coming from Colab. 

In [ ]:
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
  


english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124
def token2bucket_id(token):
    return int(_hash(token),16) % NUM_BUCKETS

# PLACE YOUR CODE HERE

def word_count(text, id):
    ''' Count the frequency of each word in `text` (tf) that is not included in 
  `all_stopwords` and return entries that will go into our posting lists. 
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs 
      for example: [("Anarchism", (12, 5)), ...]
  '''
    ps = PorterStemmer()
    word_tp = Counter()
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    ans =[]
    for token in tokens :
        token_sttem = ps.stem(token)
        if (token_sttem not in all_stopwords):
            word_tp[token_sttem] += 1
    for key in word_tp:
        ans.append((key,(id,word_tp[key])))
    return ans

def reduce_word_counts(unsorted_pl):
    ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples 
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
    return sorted(unsorted_pl)
  # YOUR CODE HERE
  #raise NotImplementedError()

def calculate_df(postings):
    ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  
    return postings.map(lambda x: (x[0],len(x[1])))


def partition_postings_and_write(postings):
    ''' A function that partitions the posting lists into buckets, writes out 
  all posting lists in a bucket to disk, and returns the posting locations for 
  each bucket. Partitioning should be done through the use of `token2bucket` 
  above. Writing to disk should use the function  `write_a_posting_list`, a 
  static method implemented in inverted_index_colab.py under the InvertedIndex 
  class. 
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and 
      offsets its posting list was written to. See `write_a_posting_list` for 
      more details.
  '''
    rdd=postings.map(lambda x : (token2bucket_id(x[0]),x))
    rdd2 = rdd.groupByKey()
    return rdd2.map(lambda x : InvertedIndex.write_a_posting_list(x,bucket_name))
  
  

In [ ]:
word_counts = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings = word_counts.groupByKey().mapValues(reduce_word_counts)
postings_filtered = postings.filter(lambda x: len(x[1])>50)
w2df = calculate_df(postings_filtered)
w2df_dict = w2df.collectAsMap()



In [ ]:
print(w2df_dict['python'])

In [ ]:
print(len(w2df_dict))

In [ ]:
def split_to_token(text,id):
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  return tokens
def doc_len_count(text, id):
  tokens = split_to_token(text,id)
  result = []
  tmp = []
  count = 0
  for i in tokens:
    if i not in all_stopwords:
      count+=1
 

    #result.append((id,(k,v)))

  return count



In [ ]:
doc_lens =  doc_text_pairs.map(lambda x: (x[1],doc_len_count(x[0], x[1])))
docs_num = doc_lens.count()

In [ ]:
idf = postings.map(lambda x: (x[0], (math.log10(docs_num/len(x[1])), x[1])))


In [ ]:
tfidf = idf.map(lambda x: (x[0], {w[0]: (x[1][0] * (w[1]/(dict_doc_lens[w[0]]))) for w in x[1][1]}))

In [ ]:
print(tfidf.collect()[168])

In [ ]:
import math

In [ ]:
def sum_all_tfidf(lst):
  new_lst=[]
  sum_all = 0
  for i in lst:
    sum_all+=i**2
  return math.sqrt(sum_all)
    

docID_rdd = tfidf.flatMap(lambda x: x[1].items()).groupByKey().mapValues(reduce_word_counts)
documents_tdidf = docID_rdd.map(lambda x :(x[0],sum_all_tfidf(x[1])))

In [ ]:
dl_rdd = doc_lens.union(documents_tdidf).reduceByKey(lambda x,y : ((x,y)))
dl_dict = dl_rdd.collectAsMap()


In [ ]:
print(len(dl_dict))

In [ ]:
#print(dl_dict[4105482])

In [ ]:
file_name = 'dl_dict_STTEMING'
base_dir = '.'
with open(Path(base_dir) / f'{file_name}.pkl','wb') as f:
	pickle.dump(dl_dict,f)

In [ ]:
# time the index creation time
#t_start = time()
# word counts map
# filtering postings and calculate df

# partition posting lists and write out
_ = partition_postings_and_write(postings_filtered).collect()
#index_const_time = time() - t_start

In [ ]:
# test index construction time
#assert index_const_time < 60*120

In [ ]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp_STEMMING'):
    if not blob.name.endswith("pickle"):
        continue
    with blob.open("rb") as f:
        posting_locs = pickle.load(f)
        for k, v in posting_locs.items():
              super_posting_locs[k].extend(v)

Putting it all together

In [ ]:
# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict
# write the global stats out
inverted.write_index('.', 'index')
# upload to gs
index_src = "index.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp_STEMMING/{index_src}'
!gsutil cp $index_src $index_dst

In [ ]:
!gsutil ls -lh $index_dst

In [15]:
client = storage.Client()
dl_dict = None
for blob in client.list_blobs('ass-3-bucket-tamar'):
    if blob.name.endswith("dl_dict_STTEMING.pkl"):
        pickle_in = blob.download_as_string()
        dl_dict = pickle.loads(pickle_in)

In [16]:
print(len(dl_dict))

6348910


In [17]:
%cd -q /home/dataproc
!ls inverted_index_stemming_gcp.py
sc.addFile("/home/dataproc/inverted_index_stemming_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

from inverted_index_stemming_gcp import *

############## test
import pickle
def open_gcp(file_name):
    client = storage.Client(file_name)
    bucket = client.bucket('ass-3-bucket-tamar')
    blob = bucket.get_blob('postings_gcp_STEMMING/' + file_name)
    return blob.open('rb')

def read_pickle(file_name):
    stream = open_gcp(file_name+".pkl")
    pick = pickle.load(stream)
    stream.close()
    return pick



#####  SOME TEST OF READING #####
# %cd -q /home/dataproc
# !ls inverted_index_gcp.py
        
# from inverted_index_gcp import InvertedIndex
# bucket_name = 'assignment3-bucket-ido' 
# # client = storage.Client()
# # blobs = client.list_blobs(bucket_name)
# # for b in blobs:
# #     if b.name == 'postings_gcp/index.pkl':
# #         with b.open("rb") as f:
# #             index = pickle.load(f)

# bucket_name = 'assignment3-bucket-ido' 
# indexName = 'index.pkl'
# index =''
# for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
#         if not blob.name.endswith(indexName):
#             continue
#         with blob.open("rb") as f:
#             index = pickle.load(f)
#             break

            
indexBody = read_pickle('index')

inverted_index_stemming_gcp.py


In [18]:
import pandas as pd
import numpy as np
import math

In [19]:
TUPLE_SIZE = 6       
TF_MASK = 2 ** 16 - 1 # Masking the 16 low bits of an integer
from contextlib import closing

def read_posting_list(inverted, w):
  with closing(MultiFileReader()) as reader:
    locs = inverted.posting_locs[w]
    b = reader.read(locs, inverted.df[w] * TUPLE_SIZE)
    posting_list = []
    for i in range(inverted.df[w]):
      doc_id = int.from_bytes(b[i*TUPLE_SIZE:i*TUPLE_SIZE+4], 'big')
      tf = int.from_bytes(b[i*TUPLE_SIZE+4:(i+1)*TUPLE_SIZE], 'big')
      posting_list.append((doc_id, tf))
    return posting_list

In [20]:
def iter_over_relevant_terms(query_to_search,index):
    candidates_list = set()
    candidates = {}
    N = len(dl_dict)
    for term in np.unique(query_to_search):
        try:
            normlized_tfidf=[]
            list_of_doc = read_posting_list(index, term)
            for doc_id,freq in list_of_doc:
                candidates_list.add(doc_id)
                normlized_tfidf.append((doc_id,(freq/dl_dict[doc_id][0])*math.log(N/index.df[term],10)))
            for doc_id,tfidf in normlized_tfidf:
                candidates[(doc_id,term)] = candidates.get((doc_id,term), 0) + tfidf
        except:
            continue
    return list(candidates_list),candidates

def cosine_similarity( query_to_search, index):
    dicti = {}
    candidates_docs_list,candidates_tfidf_dict = iter_over_relevant_terms(query_to_search,index)
    for doc in candidates_docs_list:
        mone = 0.0
        for term in np.unique(query_to_search):
            try:
                mone+=candidates_tfidf_dict[(doc,term)]
            except:
                continue
        mahane = float(dl_dict[doc][1])*math.sqrt(len(query_to_search)) #todo:documents tfidf_dict is taken from disk's memory
        dicti[doc] = mone/mahane
    return dicti

def get_top_n_docs(dicti,n=100):
    return sorted([(doc_id, score) for doc_id, score in dicti.items()], key = lambda x: x[1], reverse=True)[:n]


In [21]:
from inverted_index_stemming_gcp import *

In [37]:

from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
  
ps = PorterStemmer()

query_to_search = ['NBA']
#query_to_search = [ps.stem('where'), ps.stem('does'), ps.stem('vanilla'), ps.stem('flavoring'), ps.stem('come') ,ps.stem('from')]
print(query_to_search)
temp = cosine_similarity(query_to_search, indexBody)

top_n = get_top_n_docs(temp,100)

['NBA']


In [29]:
print(top_n)

[(48407627, 0.08187494330997451), (30796675, 0.04468197012274245), (5377129, 0.035403883214591654), (38905304, 0.03219973950725901), (8005736, 0.021061519173870467), (8778894, 0.01714705512034989), (35693777, 0.015843411869615735), (28344538, 0.015186094113402457), (13584316, 0.01481293719808463), (16274121, 0.014689267949330384), (26147929, 0.014599218452632344), (13207547, 0.014384588430903731), (36349077, 0.014018194159766196), (1967274, 0.013629889426880617), (60116836, 0.013310765874258286), (3013633, 0.012867445275187052), (57197811, 0.012119970135079913), (35584206, 0.011164118215773218), (18730910, 0.011025215330255923), (22322101, 0.010756067386546539)]


In [38]:
#need_to_be =[23862, 23329, 53672527, 21356332, 4920126, 5250192, 819149, 46448252, 83036, 88595, 18942, 696712, 2032271, 1984246, 5204237, 645111, 18384111, 3673376, 25061839, 271890, 226402, 2380213, 1179348, 15586616, 50278739, 19701, 3596573, 4225907, 19160, 1235986, 6908561, 3594951, 18805500, 5087621, 25049240, 2432299, 381782, 9603954, 390263, 317752, 38007831, 2564605, 13370873, 2403126, 17402165, 23678545, 7837468, 23954341, 11505904, 196698, 34292335, 52042, 2247376, 15858, 11322015, 13062829, 38833779, 7800160, 24193668, 440018, 54351136, 28887886, 19620, 23045823, 43003632, 746577, 1211612, 8305253, 14985517, 30796675, 51800, 964717, 6146589, 13024, 11583987, 57294217, 27471338, 5479462],
#need_to_be=[828085, 32623, 228190, 25311310, 1183972, 240668, 1808135, 10133081, 9917505, 297576, 10591, 5961225, 12897776, 4105482, 27944015, 7368588, 31591921, 431786, 64085, 12209015, 3342275, 21828, 42416319, 7082459, 48212, 529439, 7089, 704826, 228234, 225870, 3651805, 2837891, 976456, 3214240, 23054130, 54587, 8073864, 40783886, 48936348, 634940, 222266, 62149, 25342571, 167891, 241559, 4456418, 7778546, 50280699, 19886, 1188562, 249116, 6690, 35292743, 60862228, 3546873, 348499, 60088597, 113092, 180251, 7438610, 2668736, 55539, 849869, 3111871, 54840207, 64088, 2005216, 507776, 1124192, 304091, 49090939, 39727947, 1063168, 56069398, 84241, 1670565, 3631020, 566114, 1471058, 4979006, 1902624, 1613163, 654732, 937047, 16749463, 2142071, 30224834, 2208584, 34482422, 41580016, 2190209, 23139600, 24056098, 53469],
need_to_be =[22093, 16795291, 65166616, 65785063, 835946, 890793, 3921, 450389, 20455, 987153, 240940, 246185, 9000355, 5608488, 3280233, 3505049, 5958023, 72852, 8806795, 1811390, 2423824, 516570, 15392541, 72893, 412214, 278018, 12106552, 42846434, 12754503, 9807715, 4108839, 33328593, 64063961, 7215125, 1811320, 1111137, 5035602, 60483582, 9397801, 255645, 16899, 43376, 72855, 65785040, 72866, 6215230, 4987149, 72878, 16160954, 243389, 64639133, 38958735, 72858, 27196905, 38153033, 1385825, 9733533, 49926096, 4875689, 4750398, 28754077, 43569250, 22092, 72889, 59798759, 49778089, 346029, 8588996, 1956255, 52454088, 25390847, 31667631, 878666, 48695845, 72857, 459304, 27837030, 17107550, 72861, 54859596, 9195892, 6560301, 72875, 72883, 240989, 3196517, 24612090]

bad =0
good = 0
for doc_id,score in top_n:
  if doc_id in need_to_be:
    good+=1
  else:
    bad+=1
print("good",good)
print("bad",bad)

good 0
bad 0


In [39]:
client = storage.Client()
dl_dict_nostem = None
for blob in client.list_blobs('ass-3-bucket-tamar'):
    if blob.name.endswith("dl_dict.pkl"):
        pickle_in = blob.download_as_string()
        dl_dict_nostem = pickle.loads(pickle_in)

In [40]:
print(dl_dict_nostem[5376868])

(187.99299604994962, 904)


# PageRank

**YOUR TASK (10 POINTS):** Compute PageRank for the entire English Wikipedia. Use your implementation for `generate_graph` function from Colab below.

In [ ]:
# Put your `generate_graph` function here

def generate_graph(pages):
    ''' Compute the directed graph generated by wiki links.
  Parameters:
  -----------
    pages: RDD
      An RDD where each row consists of one wikipedia articles with 'id' and 
      'anchor_text'.
  Returns:
  --------
    edges: RDD
      An RDD where each row represents an edge in the directed graph created by
      the wikipedia links. The first entry should the source page id and the 
      second entry is the destination page id. No duplicates should be present. 
    vertices: RDD
      An RDD where each row represents a vetrix (node) in the directed graph 
      created by the wikipedia links. No duplicates should be present. 
  '''
  # YOUR CODE HERE
    edges = pages.flatMapValues(lambda x:x).map(lambda x:(x[0],x[1].id)).distinct()

    vertices = edges.flatMap(lambda x:x).distinct().map(lambda x: (x,))
    #raise NotImplementedError()
    return edges, vertices

In [ ]:
t_start = time()
pages_links = spark.read.parquet("gs://wikidata_preprocessed/*").select("id", "anchor_text").rdd
# construct the graph 
edges, vertices = generate_graph(pages_links)
# compute PageRank
edgesDF = edges.toDF(['src', 'dst']).repartition(124, 'src')
verticesDF = vertices.toDF(['id']).repartition(124, 'id')
g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=6)
pr = pr_results.vertices.select("id", "pagerank")
pr = pr.sort(col('pagerank').desc())
pr.repartition(1).write.csv(f'gs://{bucket_name}/pr', compression="gzip")
pr_time = time() - t_start
pr.show()

In [ ]:
# test that PageRank computaion took less than 1 hour
assert pr_time < 60*60

# Reporting

**YOUR TASK (5 points):** execute and complete the following lines to complete 
the reporting requirements for assignment #3. 

In [ ]:
# size of input data
!gsutil du -sh "gs://wikidata_preprocessed/"

In [ ]:
# size of index data
index_dst = f'gs://{bucket_name}/postings_gcp/'
!gsutil du -sh "$index_dst"

In [ ]:
# How many USD credits did you use in GCP during the course of this assignment?
cost = 3
print(f'I used {cost} USD credit during the course of this assignment')

**Bonus (10 points)** if you implement PageRank in pure PySpark, i.e. without using the GraphFrames package, AND manage to complete 10 iterations of your algorithm on the entire English Wikipedia in less than an hour. 
